In [1]:
    import pygments, mistune
    from pidgin.display import PidginDisplay

In [2]:
    lexers = {}
    formatter = pygments.formatters.html.HtmlFormatter()

In [3]:
    class HighlightRenderer(mistune.Renderer):
        def block_code(self, code, lang):
            global lexers
            lang = (lang or 'ipython').lstrip('%%')

            formatted = '' if lang in lexers else """<style>%s</style>"""%formatter.get_style_defs('.highlight')

            lexers[lang] = lexers.get(
                lang, pygments.lexers.get_lexer_by_name(lang))
            formatted += pygments.highlight(code, lexers[lang], formatter)
            return formatted

In [4]:
    renderer = HighlightRenderer()

In [5]:
    class HighlightPidginDisplay(PidginDisplay):
        def format(self): 
            return mistune.markdown(self.data, 
                                    escape=False, 
                                    parse_block_html=True, 
                                    parse_inline_html=True, 
                                   renderer=renderer)

In [6]:
    def load_ipython_extension(ip):
        %reload_ext pidgin.display
        ip.display_formatter.mimebundle_formatter.for_type(
            PidginDisplay, lambda object: HighlightPidginDisplay(
                data=object.data, environment=object.environment, ns=object.ns
            )._repr_mimebundle_())
    
    def unload_ipython_extension(ip):
        %reload_ext pidgin.display
        
    if __name__ == '__main__': load_ipython_extension(get_ipython())